In [84]:
import pandas as pd
import spacy
import textacy
import os
import logging
HOME = os.getcwd()
df = pd.read_csv(os.path.join(HOME, 'ethics_statement_frontiers_100.csv'), usecols = ['parsed'], nrows=10)
#preproc = textacy.preprocessing.make_pipeline(textacy.preprocessing.normalize.unicode,textacy.preprocessing.normalize.quotation_marks,textacy.preprocessing.normalize.whitespace)
#preproc_records = ((preproc(text), meta) for text, meta in records)
parsed_paragraphs= df.parsed.to_list()
for parsed_paragraph in parsed_paragraphs:
    parsed_paragraph.encode().decode("utf-8")
corpus = textacy.Corpus("en_core_web_sm", data=parsed_paragraphs)
print(parsed_paragraphs)


['The studies involving human participants were reviewed and approved by Ethics Committee of the Affiliated Hospital of Qingdao University School of Medicine. The patients/participants provided their written informed consent to participate in this study. Written informed consent was obtained from the individual(s) for the publication of any potentially identifiable images or data included in this article.', 'The studies involving human participants were reviewed and approved by Second affiliated hospital, school of medicine, zhejiang university. The patients/participants provided their written informed consent to participate in this study. Written informed consent was obtained from the individual(s) for the publication of any potentially identifiable images or data included in this article.', 'The studies involving human participants were reviewed and approved by the Ethics Board of Central China Normal University. The patients/participants provided their written informed consent to pa

In [85]:
from functools import partial
for doc in corpus:
    terms = list(textacy.extract.terms(doc,ngs=partial(textacy.extract.ngrams, n=2, include_pos={"NOUN", "ADJ"}),ents=partial(textacy.extract.entities, include_types={"PERSON", "ORG", "GPE", "LOC"})))
print(terms)
for term in terms:
    tokenized_docs = list((textacy.extract.terms_to_strings(terms, by="lemma")))
tokenized_docs

[Ethical review, human participants, local legislation, institutional requirements, informed consent, informed consent, identifiable images]


['ethical review',
 'human participant',
 'local legislation',
 'institutional requirement',
 'informed consent',
 'informed consent',
 'identifiable image']

In [76]:
import textacy.representations
doc_term_matrix, vocab = textacy.representations.build_doc_term_matrix(tokenized_docs, tf_type="linear", idf_type="smooth")
type(doc_term_matrix)

scipy.sparse.csr.csr_matrix

In [77]:
import textacy.tm
model = textacy.tm.TopicModel("nmf", n_topics=10)
model.fit(doc_term_matrix)

c:\users\shweata\appdata\local\programs\python\python38\lib\site-packages\sklearn\decomposition\_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


In [78]:
doc_topic_matrix = model.transform(doc_term_matrix)
doc_topic_matrix.shape

type(doc_topic_matrix)

numpy.ndarray

In [83]:
id_to_term = {id_: term for terms, id_ in vocab.items()}
#for topic_idx, terms in model.top_topic_terms(id_to_term, top_n=8):
#    print(f"topic {topic_idx}: {'   '.join(terms)}")
print(id_to_term)

{5: identifiable images, 18: identifiable images, 8: identifiable images, 9: identifiable images, 3: identifiable images, 1: identifiable images, 10: identifiable images, 0: identifiable images, 16: identifiable images, 20: identifiable images, 21: identifiable images, 19: identifiable images, 11: identifiable images, 12: identifiable images, 14: identifiable images, 13: identifiable images, 7: identifiable images, 17: identifiable images, 15: identifiable images, 6: identifiable images, 4: identifiable images, 2: identifiable images}


In [61]:
_ = model.termite_plot(doc_term_matrix, id_to_term, n_terms=30, highlight_topics=[1, 6, 9])

TypeError: Argument 'other' has incorrect type (expected spacy.tokens.span.Span, got str)

TypeError: Argument 'other' has incorrect type (expected spacy.tokens.span.Span, got str)

<Figure size 454.289x618.322 with 1 Axes>

In [92]:

import textacy.vsm
vectorizer = textacy.vsm.Vectorizer(apply_idf=True, norm=None, idf_type='standard')
doc_term_matrix = vectorizer.fit_transform(tokenized_docs)
print (doc_term_matrix.toarray())

ModuleNotFoundError: No module named 'textacy.vsm'